In [6]:
import json
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, parse_qsl

import requests
from scrapy.http import TextResponse

In [2]:
url = 'https://www.104.com.tw/jobs/search/'\
'?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=15&asc=0&page=1&mode=s'

In [3]:
# new_query_params = {'ro': '1', 'test':'test'}

def _generate_url_with_new_query_params(url, new_query_params={}):
    url_parse = urlparse(url)
    url_query = dict(parse_qsl(urlparse(url).query))
    url_query.update(new_query_params)
    new_url_query = urlencode(url_query)
    url_parse = url_parse._replace(query=new_url_query)
    new_url = urlunparse(url_parse)
    return new_url

In [4]:
with open('jsonJobCatRoot', 'r') as f:
    JSON_JOBCAT_ROOT = json.loads(f.read())
    
# JSON_JOBCAT_ROOT

In [22]:
with open('jsonAreaRoot', 'r') as f:
    JSON_AREA_ROOT = json.loads(f.read())
    
# JSON_AREA_ROOT

In [6]:
with open('jsonIndustRoot', 'r') as f:
    JSON_INDUST_ROOT = json.loads(f.read())
    
# JSON_INDUST_ROOT

In [ ]:
url_parse = urlparse(url)
url_query = dict(parse_qsl(urlparse(url).query))

jobcat_no = url_query.get('jobcat', None)
jobcat_no

In [ ]:
def _get_category_tree_by_value(d, value=None):
    if value is None:
        return d
    
    elif 'no' in d.keys():
        if d['no'] == value:
            return d
        elif 'n' in d.keys():
            for child_d in d['n']:
                d = _get_category_tree_by_value(child_d, value)
                if d is not None:
                    return d
                
    elif 'n' in d.keys():
        for child_d in d['n']:
            d = _get_category_tree_by_value(child_d, value)
            if d is not None:
                return d
    return None

category_tree = _get_category_tree_by_value(json_jobcat_root, '2007000000')
category_tree

In [ ]:
def _get_subdivided_category_no(d, category_no_list=[]):
    if 'n' in d.keys():
        for child_d in d['n']:
            category_no_list.append(child_d['no'])
        return category_no_list
        
    else:
        return None
            
_get_subdivided_category_no(category_tree)

In [62]:
#
def _get_category_no_mapping_dict(d, mapping_dict={}):
    if 'n' in d.keys():
        for child_d in d['n']:
            mapping_dict[child_d['no']] = child_d['des']
            if 'n' in child_d:
                _get_category_no_mapping_dict(child_d, mapping_dict)
    return mapping_dict

AREA_DESC_MAP_DICT = _get_category_no_mapping_dict(JSON_AREA_ROOT)

In [ ]:
url

In [ ]:
url_parse = urlparse(url)
if 'list' not in url_parse.path:
    url_parse = url_parse._replace(path=url_parse.path+'list') # to get json response of job information
new_url = urlunparse(url_parse)
new_url

In [ ]:
def _generate_json_response_url(url):
    url_parse = urlparse(url)
    if 'list' not in url_parse.path:
        url_parse = url_parse._replace(path=url_parse.path+'list') # to get json response of job information
    return urlunparse(url_parse)

In [ ]:
def _is_result_more_than_150_page(url):
    url = _generate_json_response_url(url)
    headers = {
        'Referer': url
    }
    r = requests.get(url, headers=headers)
    j = json.loads(r.text)
    return j['data']['totalPage'] >= 150

_is_result_more_than_150_page(new_url)

In [ ]:
def subdivide_url(url, urls=[]):
    
    if _is_result_more_than_150_page(url):
        
        url_parse = urlparse(url)
        url_query = dict(parse_qsl(urlparse(url).query))

        if 'area' not in url_query.keys():
            areas = _get_subdivided_category_no(JSON_AREA_ROOT)
            for area in areas:
                new_query_params = {'area': area}
                new_url = _generate_url_with_new_query_params(url, new_query_params)
                new_url = _generate_json_response_url(new_url)
                print('no area')
                print(url)
                print(new_url)
                print()
                urls = subdivide_url(new_url, urls)
        else:
            area = url_query['area']
            d = _get_category_tree_by_value(JSON_AREA_ROOT, value=area)
            areas = _get_subdivided_category_no(d)
            print(areas)
            for area in areas:
                new_query_params = {'area': area}
                new_url = _generate_url_with_new_query_params(url, new_query_params)
                new_url = _generate_json_response_url(new_url)
                print('has area')
                print(url)
                print(new_url)
                print()
                urls = subdivide_url(new_url, urls)
                
    else:
        return urls
    

subdivide_url(url)

In [7]:
#
from copy import deepcopy

def _generate_child_node_dict(tree, d={}):
    ''' According to a tree, generate child nodes for each node. 
    '''
    d = deepcopy(d)
    if 'n' in tree.keys():
        root = tree.get('no', 'root')
        nodes = []
        for node in tree['n']:
            nodes.append(node['no'])
            if 'no' in node.keys():
                d = _generate_child_node_dict(node, d)
        d[root] = nodes    
    return d

CHILD_AREA_MAP_DICT = _generate_child_node_dict(JSON_AREA_ROOT)
CHILD_JOBCAT_MAP_DICT = _generate_child_node_dict(JSON_JOBCAT_ROOT)
CHILD_INDUST_MAP_DICT = _generate_child_node_dict(JSON_INDUST_ROOT)

In [20]:
#
c = 0 #

MAX_TOTAL_PAGE = 150
DEFAULT_QUERY_PARAM = {
    # not recommended to change
    'page': 1,     # 頁數
    'mode': 's',   # s: 摘要顯示
    'order': 11,   # 11: 日期排序
    'ro': 0,       # 0: 全部職缺, 1: 全職職缺
    'asc': 0,      # 0: 降冪排序, 1: 升冪排序
}


class UrlProcessor:
    
    def __init__(self, url):
        self.url = url
        self.start_urls = []
    
        
    def start_url_getter(self):
        ''' According to url, format the url with query parameters, and subdivide url if it needs.
        '''
        url = self._formatting_url_query_param()
        urls = self._subdivide_url(url, sd_area=True)
        self.start_urls = urls
        return urls
    
    
    def all_crawl_url_getter(self):
        ''' Get all of urls that need to crawl.
        '''
        if len(self.start_urls) == 0:
            start_urls = self.start_url_getter()
        else:
            start_urls = self.start_urls
        
        
    
    
    def _formatting_url_query_param(self, query_param=DEFAULT_QUERY_PARAM):
        ''' Transform url to json-response url, and set query parameters.
        '''
        url = self.url
        url = self._transform_url_to_json_response_url(url)
        url_param = self._get_url_query_param_dict(url)
        new_url = self._generate_url_with_new_query_params(url, query_param)
        return new_url
    
    
    def _subdivide_url(self, url, urls=[], sd_area=False, sd_jobcat=False, sd_indust=False):
        ''' If the result of totlaPage is more than 150 pages, then subdivide url, 
        based on area, job category, and company industry. 
        '''
        if self._get_total_page(url) >= MAX_TOTAL_PAGE:
            if sd_area:
                url_param = self._get_url_query_param_dict(url)
                area = url_param.get('area', 'root')
                child_node = CHILD_AREA_MAP_DICT.get(area, [])
                for value in child_node:
                    new_url = self._generate_url_with_new_query_params(url, {'area': value})
                    urls = self._subdivide_url(new_url, urls, sd_area=True)
            
            if sd_jobcat:
                url_param = self._get_url_query_param_dict(url)
                jobcat = url_param.get('jobcat', 'root')
                child_node = CHILD_JOBCAT_MAP_DICT.get(jobcat, [])
                for value in child_node:
                    new_url = self._generate_url_with_new_query_params(url, {'jobcat': value})
                    urls = self._subdivide_url(new_url, urls, sd_jobcat=True)
                    
            if sd_indust:
                url_param = self._get_url_query_param_dict(url)
                indust = url_param.get('indust', 'root')
                child_node = CHILD_INDUST_MAP_DICT.get(indust, [])
                for value in child_node:
                    new_url = self._generate_url_with_new_query_params(url, {'indust': value})
                    urls = self._subdivide_url(new_url, urls, sd_indust=True)                
        else:
            urls.append(url)
                
        return urls
    
    
    @staticmethod
    def _get_total_page(url):
        ''' temp: Get the total page number of result
        '''
        global c
        c += 1
        if c > 15:
            return 1
        return 150
    
    
    @staticmethod
    def _transform_url_to_json_response_url(url):
        url_parse = urlparse(url)
        if url_parse.path.endswith('list'):
            return url
        new_path = url_parse.path + 'list'
        url_parse = url_parse._replace(path=new_path)
        new_url = urlunparse(url_parse)
        return new_url
    
    
    @staticmethod
    def _get_url_query_param_dict(url):
        url_parse = urlparse(url)
        return dict(parse_qsl(url_parse.query))
    

    @staticmethod
    def _generate_url_with_new_query_params(url, new_query_params={}):
        url_parse = urlparse(url)
        url_query = dict(parse_qsl(url_parse.query))
        url_query.update(new_query_params)
        new_url_query = urlencode(url_query)
        url_parse = url_parse._replace(query=new_url_query)
        new_url = urlunparse(url_parse)
        return new_url
        
        
url='https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=15&asc=0&page=1&mode=s'
UrlProcessor(url).start_url_getter()


['https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001002000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001003000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001004000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001005000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001006000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001007000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=11&asc=0&page=1&mode=s&area=6001008000',
 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=20070000

'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=15&asc=0&page=1&mode=s'

In [ ]:
class UrlGenerator:
    
    def __init__(self, params):
        self.params = params
    
    def generate_ajax_url(self):
        pass

In [ ]:
url_parse = urlparse(url)
url_query = dict(parse_qsl(urlparse(url).query))
url_query.update(new_query_params)
new_url_query = urlencode(url_query)
url_parse = url_parse._replace(query=new_url_query)
new_url = urlunparse(url_parse)

In [ ]:
class UrlGenerator:
    
    def __init__(self, url=None):
        self.url = url
        self.urls = []
    
    
    def undivide_url(self):
        if self.url and self._is_result_more_than_150_page(self.url):
            return self.urls
        else:
            return [self.url]
    
    
    @staticmethod
    def _generate_url_with_new_query_params(url, new_query_params={}):
        url_parse = urlparse(url)
        url_query = dict(parse_qsl(urlparse(url).query))
        url_query.update(new_query_params)
        new_url_query = urlencode(url_query)
        url_parse = url_parse._replace(query=new_url_query)
        new_url = urlunparse(url_parse)
        return new_url
    
    
    @staticmethod
    def _get_category_tree_by_value(d, value=None):
        if value is None:
            return d
        elif 'no' in d.keys():
            if d['no'] == value:
                return d
            elif 'n' in d.keys():
                for child_d in d['n']:
                    d = _get_child_category_no(child_d, value)
                    if d is not None:
                        return d
        elif 'n' in d.keys():
            for child_d in d['n']:
                d = _get_child_category_no(child_d, value)
                if d is not None:
                    return d
        return None
    
    
    @staticmethod
    def _get_subdivided_category_no(d, category_no_list=[]):
        if 'n' in d.keys():
            for child_d in d['n']:
                category_no_list.append(child_d['no'])
            return category_no_list
        else:
            return None
        
    
    @staticmethod
    def _get_category_no_mapping_dict(d, mapping_dict={}):
        if 'n' in d.keys():
            for child_d in d['n']:
                mapping_dict[child_d['no']] = child_d['des']
                if 'n' in child_d:
                    _get_category_no_mapping_dict(child_d, mapping_dict)
        return mapping_dict
    
    
    @staticmethod
    def _is_result_more_than_150_page(url):
        headers = {
            'Referer': url
        }
        r = requests.get(url, headers=headers)
        j = json.loads(r.text)
        return j['data']['totalPage'] >= 150

In [ ]:
headers = {
    'Referer': new_url
}
r = requests.get(new_url, headers=headers)
r.text

In [7]:
headers = {
    'Referer': 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=15&asc=0&page=1&mode=s',
}
url = 'https://www.104.com.tw/jobs/search/list?ro=0&jobcat=2007000000&jobcatExpansionType=0&order=15&asc=0&page=1&mode=s&area=6001000000'
r = requests.get(url, headers=headers)
response = TextResponse(r.url, body=r.text, encoding='utf-8')

In [10]:
j = json.loads(r.text)
j['status']

200

In [13]:
j['data']

{'query': {'ro': 0,
  'jobcat': '2007000000',
  'isnew': '',
  'kwop': '',
  'keyword': '',
  'jobcatExpansionType': 1,
  'excludeKeyword': '',
  'area': '6001000000',
  'indcat': '',
  'expcate': '',
  'edu': '',
  'order': 15,
  'asc': 0,
  'zone': '',
  'dep': '',
  'dis_role': '',
  'lang': '',
  'sr': '',
  'sctp': '',
  'scmin': '',
  'scmax': '',
  'scstrict': '',
  'scneg': '',
  'excludeJobKeyword': '',
  'excludeCompanyKeyword': '',
  'excludeCompanyByCustno': '',
  'excludeIndustryCat': '',
  's9': '',
  's5': '',
  'wktm': '',
  'startby': '',
  'rostatus': '',
  'page': 1,
  'wf': '',
  'jobexp': '',
  'wt': '',
  'mode': 's',
  'jobsource': '',
  'c': '',
  'custNo': '',
  'jobNo': '',
  'dist': '',
  'lat': '',
  'lon': '',
  'indArea': '',
  'newZone': '',
  'searchTempExclude': '',
  'manage': '',
  'showLatLon': ''},
 'filterDesc': {'jobcat': [{'no': 2007000000, 'des': '資訊軟體系統類'}],
  'indcat': [],
  'area': [{'no': 6001000000, 'des': '台灣地區'}],
  'dep': [],
  'excludeI

In [14]:
j['data']['filterDesc']

{'jobcat': [{'no': 2007000000, 'des': '資訊軟體系統類'}],
 'indcat': [],
 'area': [{'no': 6001000000, 'des': '台灣地區'}],
 'dep': [],
 'excludeIndustryCat': [],
 'excludeCompanyByCustno': [],
 'recommendIndcat': [{'no': '1001000000', 'des': '電子資訊 ／軟體／半導體相關業'},
  {'no': '1002000000', 'des': '一般製造業'},
  {'no': '1003000000', 'des': '批發／零售／傳直銷業'},
  {'no': '1001001000', 'des': '軟體及網路相關業'},
  {'no': '1007000000', 'des': '旅遊／休閒／運動業'}],
 'indArea': []}

In [48]:
for job in j['data']['list']:
    print(job)
    print()
    print(job['jobNo'])
    break

{'jobType': '1', 'jobNo': '11325019', 'jobName': 'Java 後端工程師', 'jobNameSnippet': 'Java 後端工程師', 'jobRole': '1', 'jobRo': '1', 'jobAddrNoDesc': '新北市新店區', 'jobAddress': '寶中路119號3樓(231)', 'description': '團隊介紹：\n主要針對平台工程與 104 產品需求，進行開發與維護的工作，\n重視團隊合作與跨部門溝通，歡迎對後端工程有興趣的夥伴加入我們！\n\n工作內容：\n．主要負責後端服務的分析與開發並持續優化\n．與前端工程師協作，討論前後端串接規則\n．需與 PM 合作，討論產品需求與了解發展方向\n\n歡迎有相關經驗人員與應屆畢業生', 'optionEdu': '專科', 'period': '00', 'periodDesc': '經歷不拘', 'applyCnt': '00019', 'applyDesc': '11~30人應徵', 'custNo': '84598349000', 'custName': '104人力銀行_一零四資訊科技股份有限公司', 'coIndustry': '', 'coIndustryDesc': '', 'salaryLow': '0000000', 'salaryHigh': '0000000', 'salaryDesc': '待遇面議', 's10': '10', 'appearDate': '20200805', 'appearDateDesc': '8/05', 'optionZone': '3410560', 'isApply': '0', 'applyDate': '', 'isSave': '0', 'descSnippet': '團隊介紹：\n主要針對平台工程與 104 產品需求，進行開發與維護的工作，\n重視團隊合作與跨部門溝通，歡迎對後端工程有興趣的夥伴加入我們！\n\n工作內容：\n．主要負責後端服務的分析與開發並持續優化\n．與前端工程師協作，討論前後端串接規則\n．需與 PM 合作，討論產品需求與了解發展方向\n\n歡迎有相關經驗人員與應屆畢業生', 'tags': ['上市上櫃', '員工750人'], 'lin

In [26]:
import re

In [21]:
url_parse = urlparse('//www.104.com.tw/job/6qqfv?jobsource=hotjob_chr')
url_parse

ParseResult(scheme='', netloc='www.104.com.tw', path='/job/6qqfv', params='', query='jobsource=hotjob_chr', fragment='')

In [30]:
job_id = re.search('job\/(.*)', url_parse.path).group(1)
job_id

'6qqfv'

In [25]:
urlparse('https://www.104.com.tw/job/ajax/content/41i1t')

ParseResult(scheme='https', netloc='www.104.com.tw', path='/job/ajax/content/41i1t', params='', query='', fragment='')

In [32]:
new_url_parse = url_parse._replace(scheme='http', path=f'/job/ajax/content/{job_id}', query='')
new_url_parse

ParseResult(scheme='http', netloc='www.104.com.tw', path='/job/ajax/content/6qqfv', params='', query='', fragment='')

In [33]:
urlunparse(new_url_parse)

'http://www.104.com.tw/job/ajax/content/6qqfv'

In [43]:
url_parse = urlparse('//www.104.com.tw/jobs/apply/analysis/6qqfv?channel=104rpt&jobsource=hotjob_chr')
url_parse

ParseResult(scheme='', netloc='www.104.com.tw', path='/jobs/apply/analysis/6qqfv', params='', query='channel=104rpt&jobsource=hotjob_chr', fragment='')

In [45]:
urlparse('https://www.104.com.tw/jb/104i/applyAnalysisToJob/all?job_no=6788513')

ParseResult(scheme='https', netloc='www.104.com.tw', path='/jb/104i/applyAnalysisToJob/all', params='', query='job_no=6788513', fragment='')

In [49]:
job_no = job['jobNo']
job_no

'11325019'

In [50]:
new_url_parse = url_parse._replace(scheme='http', path='/jb/104i/applyAnalysisToJob/all', query=f'job_no={job_no}')
new_url_parse

ParseResult(scheme='http', netloc='www.104.com.tw', path='/jb/104i/applyAnalysisToJob/all', params='', query='job_no=11325019', fragment='')

In [51]:
10%3

1

In [ ]:
class Spider:
    
    def parse(self, response):
        i = 0
        
        for r in response.xpath('//*[@id="js-job-content"]/article'):
            print(r)
            job_name_selector = r.xpath('.//div[1]/h2/a[@class="js-job-link "]')
            job_name = self._get_text_from_selector(job_name_selector)
            item['job_name'] = job_name
            print(f'>>> job_name: {job_name}')

            company_name_selector = r.xpath('.//div[1]/ul/li/a[contains(@title, "公司名")]')
            company_name = self._get_text_from_selector(company_name_selector)
            item['company_name'] = company_name
            print(f'>>> company_name: {company_name}')

            company_detailed_industry_selector = r.xpath('.//div[1]/ul[1]/li[3]')
            company_detailed_industry = self._get_text_from_selector(company_detailed_industry_selector)
            item['company_detailed_industry'] = company_detailed_industry
            print(f'>>> company_detailed_industry: {company_detailed_industry}')
            
            job_intro_tag_selector = r.xpath('.//div[1]/ul[2]/li')
            job_desc_tag = self._get_text_from_selector(job_desc_tag_selector, delimiter='、')
            item['job_desc_tag'] = job_desc_tag
            print(f'>>> job_desc_tag: {job_desc_tag}')
            
            s = r.xpath('.//div[1]/div[1]/span')
            
            print()
            
            i += 1
            if i >= 3:
                break
    
    @staticmethod
    def _get_text_from_selector(selector, delimiter=''):
        extract_result = selector.xpath('.//text()').extract()
        text = delimiter.join(extract_result).strip()
        return text
    
    
spider = Spider()
spider.parse(response)

In [ ]:
j = json.loads(r.text)
j

In [ ]:
j.keys()

In [ ]:
j['data'].keys()

In [ ]:
j['data']['count']

In [ ]:
j['data']['totalCount']

In [ ]:
j['data']['totalPage']

In [ ]:
j['data']['list'][0]

In [ ]:
with open('jsonAreaRoot', 'r') as f:
    text = f.read()

In [ ]:
len(json.loads(text)['n'])

In [2]:
import os

In [5]:
os.getcwd()

'/Users/yichen/Desktop/hr_bank/src/test'